In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.linear_model import LinearRegression
import numpy as np
import ast
from datetime import date
from pandas.stats.api import ols

In [36]:
f = '../../data/sherlock_processed2.tsv'

In [37]:
data = pd.read_csv(f, sep = '\t')

In [38]:
df = data

In [39]:
len(df)

100

In [6]:
df = df.head(100)

In [7]:
def calc_kl(p, q):
    return sum([p[i]*(np.log2(p[i]/q[i])) for i in range(len(p))])

In [8]:
timelist = df.PublishDate.drop_duplicates().tolist()
timelist = [str(i)[:7] for i in timelist]
timelist = sorted(list(set(timelist)))[2:]

In [9]:
def create_df_time(df, time):
    return df[df.PublishDate.str[:7] == time]

##Create features for regression.

###(Now regress on Chs)

###Add KL to monthly average.

In [10]:
def calc_monthly_std(df):
    gammas = []
    dis = df.TopicDistribution.tolist()
    for i in dis:
        gammas.append(ast.literal_eval(i))
        
    gammas_flat = []
    for gamma in gammas:
#         gamma = ast.literal_eval(gamma)
        for elem in gamma:
            gammas_flat.append(elem)
    ave = {}
    for n in range(40):
        t = []
        for i in gammas_flat:
            if i[0] == n:
                t.append(i[1])
        ave[n] = sum(t)/float(len(df))
    return ave.values()

In [11]:
def calc_ind_kl(td, std_month):
#     print type(std_month)
    try:
        return calc_kl(td, std_month)
    except:
        return float('nan')

In [12]:
std_gammas = {}
for time in timelist:
    df_t = create_df_time(df, time)
    std_gammas[time] = calc_monthly_std(df_t)

In [13]:
# def eval_td(cell):
#     return [i[1] for i in ast.literal_eval(cell)]

In [14]:
df['KL'] = df.apply(lambda row:calc_ind_kl([i[1] for i in ast.literal_eval(row['TopicDistribution'])], std_gammas.get(str(row['PublishDate'])[:7])), axis = 1)

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df.to_csv('../../data/sherlock_processed2.tsv', index = False, sep = '\t')

###Add # of comments/bookmarks for each chapter.

In [16]:
def sum_values(d):
    try:
        return sum(ast.literal_eval(d).values())
    except:
        return float('nan')

In [18]:
df['Comments'] = df.apply(lambda row:sum_values(row['Comments']), axis = 1)

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
df['Bookmarks'] = df.apply(lambda row:sum_values(row['Bookmarks']), axis = 1)

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
df.head(1)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Comments,CompleteDate,Fandoms,Hits,Kudos,PublishDate,Rating,Relationship,TopicDistribution,FullLength,ChapterLength,KL
0,"Implied/Referenced Rape/Non-con, Past Rape/Non...",Creator Chose Not To Use Archive Warnings,HunterusHeroicus,1,Gen,1,"John Watson, Sherlock Holmes, Mycroft Holmes, ...",2,2015-01-11,Sherlock (TV),254,5,2015-01-11,Mature,NaN,"[(0, 2.665245202558638e-05), (1, 0.02680151047...",1016,974,1.514778


###Calculate the time between publish/complete date and today

In [21]:
def date_today(cell):
    try:
        y, m, d = cell.split('-')
        return abs(date.today() - date(int(y), int(m), int(d))).days
    except:
        return float('nan')

In [22]:
df['PublishDate'] = df.apply(lambda row: date_today(row['PublishDate']), axis = 1)

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
df['CompleteDate'] = df.apply(lambda row: date_today(row['CompleteDate']), axis = 1)

/Users/jingy/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
df.head(1)

,AdditionalTags,ArchiveWarnings,Author,Bookmarks,Category,Chapters,Characters,Comments,CompleteDate,Fandoms,Hits,Kudos,PublishDate,Rating,Relationship,TopicDistribution,FullLength,ChapterLength,KL
0,"Implied/Referenced Rape/Non-con, Past Rape/Non...",Creator Chose Not To Use Archive Warnings,HunterusHeroicus,1,Gen,1,"John Watson, Sherlock Holmes, Mycroft Holmes, ...",2,505,Sherlock (TV),254,5,505,Mature,NaN,"[(0, 2.665245202558638e-05), (1, 0.02680151047...",1016,974,1.514778


In [25]:
df2 = df[['Bookmarks', 'Chapters', 'Comments', 'CompleteDate', 'Hits', 'Kudos', 'PublishDate', 'ChapterLength', 'KL']]

In [35]:
len(df)

100

In [30]:
df2 = df2.fillna(0)

In [33]:
len(df2)

100

In [31]:
df2.head()

,Bookmarks,Chapters,Comments,CompleteDate,Hits,Kudos,PublishDate,ChapterLength,KL
0,1,1,2,505,254,5,505,974,1.514778
1,0,1,0,505,157,1,505,974,1.142751
2,0,1,0,505,231,2,505,974,2.245494
3,5,1,39,505,1071,104,505,974,2.189688
4,0,1,0,505,203,1,505,974,1.859819


In [40]:
res = ols(y = df2['Kudos'], x = df2[['Chapters','CompleteDate','PublishDate','Hits']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <CompleteDate> + <PublishDate> + <Hits> + <intercept>

Number of Observations:         100
Number of Degrees of Freedom:   5

R-squared:         0.7647
Adj R-squared:     0.7548

Rmse:             51.5757

F-stat (4, 95):    77.2046, p-value:     0.0000

Degrees of Freedom: model 4, resid 95

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters    -3.1332     1.3449      -2.33     0.0219    -5.7693    -0.4971
  CompleteDate     0.0468     0.0372       1.26     0.2110    -0.0260     0.1196
   PublishDate    -0.0597     0.0588      -1.02     0.3121    -0.1749     0.0555
          Hits     0.0315     0.0041       7.75     0.0000     0.0235     0.0394
     intercept    61.2330    3

In [389]:
res = ols(y = df2['Bookmarks'], x = df2[['Chapters', 'CompleteDate', 'PublishDate', 'ChapterLength', 'KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <CompleteDate> + <PublishDate> + <ChapterLength>
             + <KL> + <intercept>

Number of Observations:         114574
Number of Degrees of Freedom:   5

R-squared:         0.1759
Adj R-squared:     0.4160

Rmse:             54.0371

F-stat (5, 114569):    -0.7326, p-value:     1.0000

Degrees of Freedom: model 5, resid 114569

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     0.4388     0.0041     106.26     0.0000     0.4307     0.4469
  CompleteDate    -0.0707     0.0013     -55.58     0.0000    -0.0732    -0.0682
   PublishDate     0.0854     0.0013      66.42     0.0000     0.0828     0.0879
 ChapterLength     0.0292        nan        nan        nan        nan  

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [377]:
res = ols(y = df2['Kudos'], x = df2[['Chapters', 'PublishDate','CompleteDate','KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <PublishDate> + <CompleteDate> + <KL> + <intercept>

Number of Observations:         132075
Number of Degrees of Freedom:   5

R-squared:         0.1147
Adj R-squared:     0.1147

Rmse:            566.4698

F-stat (4, 132070):  4277.2689, p-value:     0.0000

Degrees of Freedom: model 4, resid 132070

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters     3.8008     0.0432      88.03     0.0000     3.7161     3.8854
   PublishDate     0.7376     0.0132      55.89     0.0000     0.7118     0.7635
  CompleteDate    -0.6166     0.0131     -47.24     0.0000    -0.6422    -0.5910
            KL   -49.1199     1.7183     -28.59     0.0000   -52.4877   -45.7520
     intercept   146.

In [355]:
res = ols(y = df2['Hits'], x = df2[['Chapters', 'CompleteDate', 'PublishDate', 'ChapterLength', 'KL']])
res


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Chapters> + <CompleteDate> + <PublishDate> + <ChapterLength>
             + <KL> + <intercept>

Number of Observations:         131300
Number of Degrees of Freedom:   5

R-squared:         0.1473
Adj R-squared:     0.2346

Rmse:          20534.5567

F-stat (5, 131295):    -1.6853, p-value:     1.0000

Degrees of Freedom: model 5, resid 131295

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      Chapters   143.2248     1.5535      92.20     0.0000   140.1799   146.2696
  CompleteDate   -29.8870     0.4706     -63.51     0.0000   -30.8094   -28.9647
   PublishDate    36.4067     0.4743      76.77     0.0000    35.4772    37.3363
 ChapterLength     1.3914        nan        nan        nan        nan  